<a href="https://colab.research.google.com/github/s23912/pad/blob/main/PAD_LAB_07_ZADANIE_DOMOWE_pawel_ungier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
path ='/content/drive/MyDrive/PAD/PAD_07/Zadanie_domowe/SMSSpamCollection.txt'
df = pd.read_csv(path, sep='\t', header=None)
df.columns=['label', 'text']
df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
df.groupby(by=['label']).size()

label
ham     4825
spam     747
dtype: int64

#a)	Dokonaj klasyfikacji sms-ów ze względu na to, czy są spamem czy nie b)	Jako dane wejścia wykorzystaj wagi TF-IDF

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer



def clean_text(x):
    x = x.lower()
    x = x.replace('.','')
    x = x.replace(',','')
    x = x.replace(':','')
    x = x.replace(';','')
    x = x.replace('!','')
    x = x.replace('?','')
    x = x.replace('<br>','')
    x = x.replace('<br />','')
    x = ' '.join([word for word in x.split() if word.isalpha()])
    return x

df['text']= df['text'].apply(clean_text)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])
vectorizer.get_feature_names_out()
print(X.shape)

(5572, 7751)


In [ ]:
X.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
vectorizer.get_feature_names() #kolekcja wszystkich słów z korpusu
pd.DataFrame(X.todense(), columns = vectorizer.get_feature_names()) #reprezentacja w formie ramki danych

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,aah,aaniye,aaooooright,aathilove,aathiwhere,ab,abbey,abdomen,abeg,abelu,aberdeen,abi,ability,abiola,abj,able,abnormally,about,aboutas,above,abroad,absence,absolutely,abstract,abt,abta,aburo,abuse,abusers,ac,academic,acc,accent,accenture,accept,access,accessible,accidant,accident,...,youphone,your,youre,yourinclusive,yourjob,yours,yourself,youso,youthats,youto,youuuuu,youwanna,youwhen,yoville,yowifes,yoyyooo,yr,yrs,ystrdayice,yummmm,yummy,yun,yunny,yuo,yuou,yup,yupz,ywhere,zac,zealand,zebra,zed,zeros,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#c)	Dodaj informacje o sentymencie smsa do modelu klasyfikacyjnego

In [ ]:
targets = df['label']

#d)	Dodaj informacę o proporcji czasowników („VERB”), rzeczowników („NOUN”) i przymiotników („ADJ”)

In [ ]:
!python -m pip install spacy==2.3.2 -q
!python -m spacy download en_core_web_sm


     |████████████████████████████████| 9.9 MB 25.6 MB/s 
     |████████████████████████████████| 2.1 MB 48.1 MB/s 
     |████████████████████████████████| 12.0 MB 25.6 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047105 sha256=e57585cc27b8209a4d36a81092b1a819d6a78f04a37e8839a5fcbe92dd9cb653
  Stored in directory: /tmp/pip-ephem-wheel-cache-dfr2ygte/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')


In [ ]:
def count_nouns (sentence): #rzeczownik
  tokens=nlp(sentence)
  poses = [token.pos_ for token in tokens if token.pos_=='NOUN']
  return len(poses)


def count_verb (sentence): #czasownik
  tokens=nlp(sentence)
  poses = [token.pos_ for token in tokens if token.pos_=='VERB']
  return len(poses)

def count_adj (sentence): # przymiotnik
  tokens=nlp(sentence)
  poses = [token.pos_ for token in tokens if token.pos_=='ADJ']
  return len(poses)

In [ ]:
df['nouns'] = df['text'].apply(count_nouns)

In [ ]:
df['verbs'] = df['text'].apply(count_verb)

In [ ]:
df['adj'] = df['text'].apply(count_adj)

In [ ]:
X.shape
df['nouns'].shape

(5572,)

In [ ]:
import numpy as np
from scipy.sparse import hstack
#https://stackoverflow.com/questions/41937786/add-column-to-a-sparse-matrix

#df['nouns']
#df['verbs']
#df['adj']

def to_sparse_matrix(column):
    return np.array(column)[:, None]

df_N = to_sparse_matrix(df['nouns'])
df_V = to_sparse_matrix(df['verbs'])
df_A = to_sparse_matrix(df['adj'])

X = hstack((X, df_N,df_V, df_A ))

In [ ]:
X.A
# trzy osttanie kolumny to liczba rzeczowników, czasowników oraz przymiotników w zdaniu

array([[0., 0., 0., ..., 2., 2., 2.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 5., 5., 2.],
       ...,
       [0., 0., 0., ..., 3., 0., 2.],
       [0., 0., 0., ..., 3., 3., 3.],
       [0., 0., 0., ..., 2., 1., 0.]])

In [ ]:
#sprawdzamy czy poprawnie podaliśmy liczbę 
print(df.loc[77, 'text'])
print(X.A[77])

i like you peoples very but am very shy pa
[0. 0. 0. ... 1. 1. 1.]


In [ ]:
[token.pos_ for token in nlp(df.loc[77,'text'])]
# noun -> people
# verb -> like
# adj -> shy
# czyli poprawnie są podane wartości

['PRON', 'VERB', 'PRON', 'NOUN', 'ADV', 'CCONJ', 'AUX', 'ADV', 'ADJ', 'PROPN']

#e)	Wykorzystaj 3 znane Ci algorytmy klasyfikacyjne.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size = 0.25, random_state = 42)

Logistic_r = LogisticRegression(random_state=0)
Logistic_r.fit(X_train, y_train)
prediction_lr = Logistic_r.predict(X_test)
accuracy_score(y_test, prediction_lr)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9691313711414213

In [ ]:
from sklearn.naive_bayes import GaussianNB
naive_b = GaussianNB()
naive_b.fit(X_train.A, y_train)
prediction_nb = naive_b.predict(X_test.A)
accuracy_score(y_test, prediction_nb)

0.882986360373295

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train.A, y_train)
prediction_knn = knn.predict(X_test.A)
accuracy_score(y_test, prediction_knn)

0.8994974874371859